<a href="https://colab.research.google.com/github/juanjoseariass/Anal-tica-4/blob/main/TechSalesReport_JuanJoseArias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerías

In [ ]:
import numpy as np
import pandas as pd

#para implementación y evalución del modelo
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#Para carga automatica de los archivos desde el drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Carga de Datos

In [ ]:
nxl="/content/drive/MyDrive/2. TechSalesReport_Int_L.xlsx"
XDB1=pd.read_excel(nxl,sheet_name="TechSales_Reps")
XDB = XDB1.copy()
XDB['Business'] = XDB['Business'].replace({'Software': 1, 'Hardware': 0}) #Cambiar los valores de la columna "Business" por ceros y unos
XDB['College'] = XDB['College'].replace({'Yes': 1, 'No': 0}) #Cambiar los valores de la columna "College" por ceros y unos
XDB.head(100) #Solo se muestran 100 datos

<ipython-input-2-8f348e859a8b>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  XDB['Business'] = XDB['Business'].replace({'Software': 1, 'Hardware': 0}) #Cambiar los valores de la columna "Business" por ceros y unos
<ipython-input-2-8f348e859a8b>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  XDB['College'] = XDB['College'].replace({'Yes': 1, 'No': 0}) #Cambiar los valores de la columna "College" por ceros y unos


,Sales_Rep,Business,Age,Female,Years,College,Personality,Certificates,Feedback,Salary,NPS
0,1,0,59,1,2,1,Diplomat,1,2.01,70200,5
1,2,0,52,0,10,1,Diplomat,4,3.64,133000,10
2,3,1,47,1,1,1,Explorer,1,3.88,52600,8
3,4,0,61,0,2,1,Diplomat,3,2.70,96000,6
4,5,1,39,0,1,0,Diplomat,5,3.44,122000,7
...,...,...,...,...,...,...,...,...,...,...,...
95,96,1,27,0,2,1,Analyst,4,1.58,41000,4
96,97,0,28,0,2,1,Sentinel,3,2.70,55000,4
97,98,0,46,1,1,1,Sentinel,6,3.34,86800,9
98,99,0,40,0,1,0,Sentinel,5,1.95,49000,6


# Descripción de Variables

**Business**: Tipo de negocio en el que trabaja el empleado, con valores "Software" o "Hardware".\
**Age**: Edad del empleado en años.\
**Female**: Indicador binario de género (1 = mujer, 0 = hombre).\
**Years**: Años de experiencia del empleado en la empresa.\
**College**: Indica si el empleado asistió a la universidad ("Yes" para sí, "No" para no).\
**Certificates**: Número de certificaciones obtenidas por el empleado.\
**Feedback**: Calificación de retroalimentación recibida, representada con valores decimales.\
**Salary**: Salario anual del empleado (probablemente en dólares).\
**NPS**: Net Promoter Score, una métrica de satisfacción y lealtad del cliente, en una escala de 0 a 10.


# Caracterización de la Base de Datos

In [ ]:
total_registros = len(XDB1)
print("Total de personas:", total_registros)
valores = XDB1['College'].value_counts()
print("Personas con título universitario:", valores.iloc[0])
print("Personas sin título universitario:", valores.iloc[1])

Total de personas: 21990
Personas con título universitario: 17520
Personas sin título universitario: 4470


# Estructura del Árbol de Decisión

In [ ]:
XD=XDB[['Business','Age','Female','Years','Certificates','Feedback','Salary','NPS']]
yd=XDB[['College']]
mar=DecisionTreeClassifier(criterion='gini',max_depth=4)
mar.fit(XD,yd)

#Se procede con el despliegue del árbol
from sklearn.tree import export_graphviz #Exportar
from pydotplus import graph_from_dot_data #Crear gráfico

vs=['Business','Age','Female','Years','Certificates','Feedback','Salary','NPS']

dot_data=export_graphviz(mar,feature_names=vs)
graph=graph_from_dot_data(dot_data)
graph.write_png('1. ArbolCollege.png')

tree = mar.tree_
nodos_hoja = np.where(tree.children_left == -1)[0]
nodos_puros = sum(tree.impurity[nodo] == 0 for nodo in nodos_hoja)

print(f"Número de nodos puros: {nodos_puros}")

Número de nodos puros: 0


# Desempeño del Modelo

In [ ]:
ydp=mar.predict(XD)
cm=confusion_matrix(yd,ydp)
print(cm)


VN=cm[0,0]; FP=cm[0,1]; FN=cm[1,0]; VP=cm[1,1]

Exactitud=(VP+VN)/(VP+VN+FP+FN) #Funcionamiento general del modelo
print('Exactitud:',Exactitud)
Sensibilidad=VP/(VP+FN) #Cómo se comportó el modelo con los que poseen título universitario
print('Sensibilidad:',Sensibilidad)
Especificidad=VN/(VN+FP) #Cómo se comportó el modelo con los que no poseen título universitario
print('Especificidad:',Especificidad)
Precision=VP/(VP+FP) #Cuál es el acierto clasificando los que poseen título universitario
print('Precision:',Precision)
TasaError=(FP+FN)/(VP+VN+FP+FN) #Equivocación general
print('TasaError:',TasaError)
PredNeg=VN/(VN+FN) #Cómo funciona el modelo prediciendo que los empleados no poseen título universitario
print('NoCollege:',PredNeg)

[[  114  4356]
 [   69 17451]]
Exactitud: 0.7987721691678036
Sensibilidad: 0.9960616438356165
Especificidad: 0.025503355704697986
Precision: 0.8002476269087908
TasaError: 0.20122783083219645
NoCollege: 0.6229508196721312


# Pronóstico

In [ ]:
#Se toma la totalidad de los solicitante
XDB2=pd.read_excel(nxl,sheet_name="Pronóstico")
XDB3 = XDB2.copy()
XDB3['Business'] = XDB3['Business'].replace({'Software': 1, 'Hardware': 0}) #Cambiar los valores de la columna "Business" por ceros y unos

#Se hace el propósito de la PreApr para estas personas
XD2=XDB3[['Business','Age','Female','Years','Certificates','Feedback','Salary','NPS']]

#Se hace el pronostico para estas personas
ydp2=mar.predict(XD2)
print("Los pronósticos del título universitario para las personas de la base de datos de pronóstico son:\n",ydp2)

#crear el informe con la PreApr
dfp=pd.DataFrame(np.column_stack((XD2,ydp2)))
dfp.columns=['Business','Age','Female','Years','Certificates','Feedback','Salary','NPS','College']
dfp.to_excel('PronósticoCollege.xlsx')

print("Los pronosticados sin título universitario son:",len(np.where((ydp2==0))[0]))
print("Los pronosticados con título universitario son:",len(np.where((ydp2==1))[0]))

Los pronósticos del título universitario para las personas de la base de datos de pronóstico son:
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Los pronosticados sin título universitario son: 0
Los pronosticados con título universitario son: 16


<ipython-input-12-17d03692d359>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  XDB3['Business'] = XDB3['Business'].replace({'Software': 1, 'Hardware': 0}) #Cambiar los valores de la columna "Business" por ceros y unos
